In [70]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [71]:
dataset=pd.read_csv("insurance_pre.csv")
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [72]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [73]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [74]:
independent=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [75]:
dependent=dataset[['charges']]
dependent 

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [76]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent, dependent, test_size=0.30,random_state=0)

In [77]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)
x_train

array([[-1.5330973 , -0.40713453, -0.89833872, -0.97676557, -0.50466988],
       [-0.03364163,  0.32855417, -0.89833872, -0.97676557, -0.50466988],
       [ 0.89459283,  2.56690911,  3.25603402, -0.97676557, -0.50466988],
       ...,
       [ 0.03776102, -0.91016269, -0.89833872,  1.02378711, -0.50466988],
       [-1.46169465,  0.76659782, -0.89833872,  1.02378711, -0.50466988],
       [-0.46205754, -1.96596021, -0.06746417, -0.97676557, -0.50466988]])

In [78]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
param_grid = {"kernel":["rfb","poly","sigmoid","linear"],
             "C":[10,100,1000,2000,3000],"gamma":["auto","scale"]}

grid = GridSearchCV(SVR(), param_grid, refit = True, verbose = 3,n_jobs=-1)
#fitting the model for grid search
grid.fit(x_train, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


C:\Users\Admin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
50 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Admin\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 255, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "C:\Users\Admin\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 315, in _dense_fit
    ) = libsvm.fit(
  File "sklearn\svm\_libsvm.pyx"

GridSearchCV(estimator=SVR(), n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'gamma': ['auto', 'scale'],
                         'kernel': ['rfb', 'poly', 'sigmoid', 'linear']},
             verbose=3)

In [79]:
# print best parameter after tuning
# print(grid.best_params)
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(x_test)
#print classification report
from sklearn.metrics import r2_score
r_score=r2_score(y_test,grid_predictions)

print("The R_score valu for best parameter {}:".format(grid.best_params_),r_score)

The R_score valu for best parameter {'C': 3000, 'gamma': 'scale', 'kernel': 'poly'}: 0.8598930084494356


In [80]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001774,0.000390,0.000000,0.000000e+00,10,auto,rfb,"{'C': 10, 'gamma': 'auto', 'kernel': 'rfb'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,40
1,0.043242,0.006748,0.007580,4.889087e-04,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",0.056274,0.069532,-0.045601,-0.025079,-0.049592,0.001107,0.051309,24
2,0.048448,0.009020,0.003124,6.248283e-03,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.049905,0.075905,-0.046585,-0.041004,-0.046507,-0.001657,0.053391,26
3,0.029401,0.002355,0.009372,7.652592e-03,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.377969,0.479601,0.317872,0.337979,0.324422,0.367569,0.059777,19
4,0.006261,0.007669,0.000000,0.000000e+00,10,scale,rfb,"{'C': 10, 'gamma': 'scale', 'kernel': 'rfb'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,39
5,0.034366,0.006249,0.009373,7.653371e-03,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",0.054964,0.071297,-0.046513,-0.024157,-0.049652,0.001188,0.051594,23
6,0.052685,0.008797,0.010905,5.751493e-03,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.049644,0.076323,-0.046798,-0.040824,-0.046521,-0.001635,0.053474,25
7,0.041007,0.008034,0.007579,5.007049e-03,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.377969,0.479601,0.317872,0.337979,0.324422,0.367569,0.059777,19
8,0.000000,0.000000,0.000000,0.000000e+00,100,auto,rfb,"{'C': 100, 'gamma': 'auto', 'kernel': 'rfb'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,36
9,0.041702,0.007148,0.011456,3.427328e-03,100,auto,poly,"{'C': 100, 'gamma': 'auto', 'kernel': 'poly'}",0.540643,0.575051,0.474839,0.535730,0.424640,0.510181,0.053582,15


In [83]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI"))
children_input=float(input("children:"))
sex_male_input=int(input("sex Male 0 or 1:"))
smoker_yes_input=int(input("smoker yes 0 or 1:"))

Age:32
BMI25
children:1
sex Male 0 or 1:2
smoker yes 0 or 1:1


In [84]:
Future_prediction=grid.predict([[age_input,age_input,children_input,sex_male_input,smoker_yes_input]])
print("Future_prediction={}".format(Future_prediction))

Future_prediction=[4903191.33956351]
